In [1]:
import tensorflow as tf

In [2]:
import numpy as np
import cv2 as cv
import argparse
from PIL import Image, ImageEnhance, ImageDraw 
import matplotlib.pyplot as plt
import os

In [3]:
def ROI(frame):
    #enhancer = ImageEnhance.Contrast(frame)
    #img = enhancer.enhance(0.7)
    face_classifier = cv.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv.cvtColor(frame, cv.COLOR_RGB2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.1, 5)
    x=0
    y=0
    w=0
    h=0
    max_area = 0
    for face in faces:
        x_t, y_t, w_t, h_t = face
        if w_t*h_t>max_area:
            max_area = w_t*h_t
            x,y,w,h = face
    #print(faces)
    
    return x,y,w,h

In [4]:
import keras
model = keras.models.load_model("ResNet_Model1")

In [5]:
emotion = {
    '0' : 'Afraid',
    '1' : 'Angry',
    '2' : 'Disgust',
    '3' : 'Happy',
    '4' : 'Nervous',
    '5' : 'Sad',
    '6' : 'Surprise'
}

In [7]:
#def using_cam():
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    #cv.imshow("Input",frame)
    frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    x,y,w,h = ROI(frame)
    cv.rectangle(frame, (x,y), (x+w, y+h), (127, 0, 255), 2)
    croped_image = frame[x:x+w, y:y+h]
    #croped_image = frame
    #croped_image = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    #cv.imshow('ROI', croped_image)
    if(cv.waitKey(2) == 13 & 0xFF):
        break
        
        
    img = Image.fromarray(croped_image)
    img.save("Pic.jpg")
    img = keras.preprocessing.image.load_img(
        'Pic.jpg', target_size=(180, 180)
    )
    img_array = keras.preprocessing.image.img_to_array(img)
    img_array = tf.expand_dims(img_array, 0) # Create a batch
    predictions = model.predict(img_array)
    score = np.array(tf.nn.softmax(predictions[0]))
        
    font = cv.FONT_HERSHEY_SIMPLEX
    fontScale = 0.5
    thickness = 2
    color = (127, 0, 255)
    a = 50
    b = 50
    pos = (a, b)
    li1 = [0,1,2,3,4,5,6]
    print(score)
    for emo in li1:
        image = cv.putText(frame, emotion[str(emo)]+" : "+str(score[emo]), pos, font, fontScale, color, thickness, cv.LINE_AA)
        b+=50
        pos = (a,b)
        
        #print(emotion[str(np.argmax(score))])
    image = cv.putText(frame, emotion[str(np.argmax(score))], (150,50), font, 2, color, thickness, cv.LINE_AA)
    cv.imshow("Final", image)
cap.release()
cv.destroyAllWindows()

[0.1260345  0.16490269 0.14520238 0.12604743 0.12857988 0.18411887
 0.12511425]
[0.12206425 0.16776258 0.12806734 0.12446259 0.12599    0.20884848
 0.12280472]
[0.12541232 0.19196162 0.13221216 0.12424633 0.12535672 0.17728238
 0.12352852]
[0.13761637 0.13733728 0.15096918 0.12460575 0.14326143 0.14424315
 0.16196692]
[0.1275056  0.13992597 0.13991025 0.12520021 0.14736769 0.16523966
 0.15485077]
[0.12734501 0.15357022 0.15107714 0.12670861 0.13469873 0.16873406
 0.13786626]
[0.12333067 0.17253047 0.1371007  0.12422652 0.12536539 0.19421881
 0.12322742]
[0.13791971 0.12194486 0.13186523 0.12173751 0.13341914 0.12684162
 0.22627194]
[0.15590225 0.13242681 0.19383378 0.12855317 0.12368079 0.14164403
 0.12395917]
[0.12775487 0.14290707 0.16598566 0.13132416 0.1312167  0.17353746
 0.12727408]
[0.12822212 0.17750798 0.15923648 0.12383202 0.12386844 0.16341601
 0.12391699]
[0.12527624 0.11871528 0.12372372 0.11882728 0.13090213 0.11919499
 0.2633604 ]
[0.2845569  0.11847003 0.12244143 0.1180

[0.12540287 0.12162639 0.1243538  0.12090399 0.11939071 0.12085894
 0.26746336]
[0.16746831 0.12373178 0.12374687 0.12319858 0.12156448 0.12493557
 0.21535443]
[0.12801512 0.19630009 0.1700562  0.12463402 0.12209497 0.13595106
 0.1229485 ]
[0.14542936 0.13238294 0.13263863 0.1241395  0.12354225 0.13094558
 0.21092172]
[0.13429354 0.14034891 0.1533786  0.12712078 0.12732057 0.13122611
 0.18631142]
[0.12786265 0.14538041 0.2136474  0.12880145 0.12436576 0.12867619
 0.1312661 ]
[0.12172978 0.13404806 0.2346064  0.12469843 0.12156025 0.13751449
 0.12584265]
[0.12470921 0.18247008 0.16796415 0.1251851  0.12310185 0.15023793
 0.12633164]
[0.16095054 0.1707121  0.15351145 0.12592119 0.12513936 0.13727616
 0.12648922]
[0.12971859 0.1980856  0.16232258 0.13122022 0.12803498 0.12361967
 0.12699836]
[0.12705706 0.18993358 0.17260896 0.12766469 0.13040493 0.12375834
 0.1285725 ]


ValueError: tile cannot extend outside image